In [ ]:
import cv2
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output
import time

from tiki.mini import TikiMini

In [ ]:
tiki = TikiMini()

In [ ]:
# Hyper Parameters
lower_green = np.array([67, 23, 101])
upper_green = np.array([80, 255, 168])

lower_line = np.array([0, 0, 0])
upper_line = np.array([360, 255, 50])

rho = 1
theta = np.pi / 180
threshold = 80
min_line_length = 50
max_line_gap = 10
angle_threshold = 30

In [3]:
class Wrapper:
    def __init__(self, save_name=None):
        self.cap = cv2.VideoCapture(
            "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, framerate=15/1, format=NV12 ! "
            "nvvidconv flip-method=2 ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
        )
    def __enter__(self):
        print('entered')
        return self.cap
    def __exit__(self,a,b,c):
        self.cap.release()
        print('released')

In [ ]:
def detect_lines(image):
    
    crop_img = image[240:480, 150:480].copy()

    hsv_image = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    color_mask = cv2.inRange(hsv_image, lower_line, upper_line)
    masked_image = cv2.bitwise_and(crop_img, crop_img, mask=color_mask)
    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)
    lines, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return crop_img, lines

In [ ]:
def adjust_wheel_speed(steering_angle, base_speed=20):
    left_speed = base_speed
    right_speed = base_speed
    
    if steering_angle > 0:  # Turn right
        left_speed += steering_angle * base_speed
        right_speed -= steering_angle * base_speed

    else:  # Turn left
        left_speed += abs(steering_angle) * base_speed
        right_speed -= abs(steering_angle) * base_speed
    
    return int(left_speed), int(right_speed)

In [ ]:
# 카메라에서 실시간 이미지 출력하는 함수
def main():
    video_widget = widgets.Image(format='jpeg')
    tiki.set_motor_mode(tiki.MOTOR_MODE_PID)
    with Wrapper() as cap:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            crop_img, lines = detect_lines(frame, rho, theta, threshold, min_line_length, max_line_gap)
            
            if len(lines) > 0:
                
                c = max(lines, key=cv2.contourArea)
                M = cv2.moments(c)
                if M['m00'] != 0:
                    cx = int(M['m10']/M['m00'])
                    cy = int(M['m01']/M['m00'])

                cv2.line(crop_img,(cx,0),(cx,640),(255,0,0),1)
                cv2.line(crop_img,(0,cy),(480,cy),(255,0,0),1)
                cv2.drawContours(crop_img, lines, -1, (0,255,0), 1)

                print(cx)


            frame[240:480, 150:480] = crop_img
            cv2.putText(frame, f"cx: {cx}s", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            
            _, buffer = cv2.imencode('.jpg', frame)
            video_widget.value = buffer.tobytes()
            
            clear_output(wait=True)
            display(video_widget)
            time.sleep(0.03)  # 프레임 간 간격

if __name__ == '__main__':
    main()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

released


KeyboardInterrupt: 